# ERV differentiation

Two main research questions:
1. Are ERV's differentially expressed in SCC versus AC?
2. Are ERV's involved in immune surveillance in lung cancer

We have to 
* identify ERV's in Adeno and SCC
* identify granzy mand perforin expression per subtype
* identify IFN gene expression per subtype
* silencing of ERV by methylation in normal tissue



In [1]:
import vaex as vx
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import scipy as sc

import os
import sys
from collections import Counter
from collections import namedtuple

In [38]:
df = pd.read_csv("/media/koekiemonster/DATA-FAST/genetic_expression/hackathon_2/Lung/Lung_miRNA.txt",
                               sep="\t", engine="python")

In [39]:
df.shape

(2281, 881)

In [40]:
df.head()

,MIMATID,Name,Chr,Start,Stop,Strand,TCGA-NJ-A4YP-01,TCGA-86-8278-01,TCGA-62-A470-01,TCGA-44-6778-01,...,TCGA-58-A46L-01,TCGA-77-8144-01,TCGA-J1-A4AH-01,TCGA-56-7580-01,TCGA-63-A5MY-01,TCGA-33-AASL-01,TCGA-85-A512-01,TCGA-85-8354-01,TCGA-O2-A5IB-01,TCGA-77-7335-01
0,MIMAT0000062,hsa-let-7a-5p,chr11,122146568.0,122146589.0,-,14.939725,14.771870,14.332978,14.454362,...,14.096087,14.524829,14.274734,14.175049,13.559968,14.466466,14.721147,14.785444,14.373206,14.863538
1,MIMAT0000063,hsa-let-7b-5p,chr22,46113691.0,46113712.0,+,14.327167,14.010682,13.384376,13.834904,...,13.367341,13.776121,12.944346,14.013856,11.129973,12.926317,13.626191,13.998034,14.149870,14.183938
2,MIMAT0000064,hsa-let-7c-5p,chr21,16539838.0,16539859.0,+,9.595746,9.645857,9.848584,10.302141,...,9.819101,9.049123,7.965848,9.828557,9.559888,9.058416,9.697313,10.208731,12.793410,11.255997
3,MIMAT0000065,hsa-let-7d-5p,chr9,94178841.0,94178862.0,+,7.405218,6.728663,7.684090,7.625472,...,7.430944,7.570229,7.675079,7.814177,7.471836,8.241641,8.269299,8.885391,8.184215,8.319972
4,MIMAT0000066,hsa-let-7e-5p,chr19,51692793.0,51692814.0,+,9.040771,9.723797,9.878375,9.523750,...,8.877653,9.226451,9.819179,10.071820,7.898879,10.200385,8.910055,10.134812,10.163920,9.671110


In [41]:
# write to HDF5
vaex_df = vx.from_pandas(df, copy_index=False)
vaex_df.export_hdf5(path="/media/koekiemonster/DATA-FAST/genetic_expression/hackathon_2/Lung/Lung_miRNA.hdf5", progress=True)